In [1]:
import numpy as np
import pandas as pd
from dijkstra import DijkstraSPF, Graph
from sklearn.neighbors import BallTree
import math
import csv

In [2]:
data_file = './data/worldcities_ascii.csv'
with open(data_file, 'r') as f:
    reader = csv.reader(f)
    all_lines = []
    for row in reader:
        all_lines.append(row)

In [3]:
newLines = []
for line in all_lines:
    newLine = []
    for index, field in enumerate(line):
        if ("." in field) and (index == 5): #Toglie il '.' nei valori della popolazione
            field = field.replace(".", "")
        field = field.replace("Korea, South", "South Korea")
        field = field.replace("Korea, North", "North Korea")
        field = field.replace("Gambia, The", "The Gambia")
        field = field.replace("Micronesia, Federated States Of", "Federated States Of Micronesia")
        field = field.replace("Bahamas, The", "The Bahamas")
        field = field.replace("Saint Helena, Ascension, And Tristan Da Cunha", "Saint Helena, Ascension and Tristan da Cunha")
        field = field.replace("Islamorada, Village of Islands", "Village of Islands Islamorada")
        if index == 6:
            field = field.replace("\n", "")
        newLine.append(field)
    newLines.append(newLine)

#Lo esporto in caso mi servisse in altri progetti
df = pd.DataFrame(newLines[1:], columns=newLines[0])
df["population"].replace({"": 0}, inplace= True) #Correggo alcuni valori che non hanno la popolazione con 0 anche se sarebbe più corretto lasciare null/None
df.to_csv("./data/worldcities_preprocessed.csv", index=False)
df

,city,lat,lng,country,iso3,population,id
0,Tokyo,35.6897,139.6922,Japan,JPN,37977000,1392685764
1,Jakarta,-6.2146,106.8451,Indonesia,IDN,34540000,1360771077
2,Delhi,28.66,77.23,India,IND,29617000,1356872604
3,Mumbai,18.9667,72.8333,India,IND,23355000,1356226629
4,Manila,14.5958,120.9772,Philippines,PHL,23088000,1608618140
...,...,...,...,...,...,...,...
26564,Nord,81.7166,-17.8,Greenland,GRL,10,1304217709
26565,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,10,1304206491
26566,Cheremoshna,51.3894,30.0989,Ukraine,UKR,0,1804043438
26567,Ambarchik,69.651,162.3336,Russia,RUS,0,1643739159


## Mi costruisco il grafo con un balltree

Guarda interactive maps per far vedere risultato se puoi utilizzare direttamente quelle con rise o simili

In [4]:
coords = [np.array([float(c.lat), float(c.lng)])*math.pi/180 for _, c in df.iterrows()]

In [5]:
tree = BallTree(coords, leaf_size=3, metric="haversine")
tree.query(coords[284:285], k=4)

(array([[0.        , 0.0025749 , 0.00269627, 0.00408044]]),
 array([[  284,   999, 11417,  3118]], dtype=int64))

In [6]:
def distanceTime(start, end, neighbour_pos: int):
    if neighbour_pos == 0:
        time = 2
    elif neighbour_pos == 1:
        time = 4
    elif neighbour_pos == 2:
        time = 8
    if start.iso3 != end.iso3:
        time += 2
    if float(end.population) > 200000:
        time += 2
    return time

In [7]:
graph = Graph()
for i in df.index:
    n = tree.query(coords[i:i+1], k=4)[1][0][1:]
    
    graph.add_edge(i, n[0], distanceTime(df.iloc[i], df.iloc[n[0]], 0))
    graph.add_edge(i, n[1], distanceTime(df.iloc[i], df.iloc[n[1]], 1))
    graph.add_edge(i, n[2], distanceTime(df.iloc[i], df.iloc[n[2]], 2))

In [8]:
startNode = 3175 #Città di partenza
dijkstra = DijkstraSPF(graph, startNode)
print("%-5s %-5s %-10s" % ("label", "distance", "path"))
for endNode in df.index:
    dist = dijkstra.get_distance(endNode)
    if dist != float('inf'):
        print("%-5s %-8d %-10s" % (endNode, dist, dijkstra.get_path(endNode)))
        break

label distance path      
2     108      [3175, 19004, 14093, 14599, 10956, 831, 14757, 2913, 253, 1810, 2275, 10809, 3809, 627, 15303, 3178, 2520, 2709, 337, 2]


# Map ipyleaflet

In [20]:
from ipyleaflet import Map, basemaps, Marker, Polyline
import ipywidgets as widgets

## All cities neighbours

In [38]:
def draw_marker(city):
    marker = Marker(location=(city.lat, city.lng), draggable=False, title=city.city)
    #marker.on_click(lambda **a: add_marker(df.iloc[193]))
    m.add_layer(marker)
    
def draw_line(coords):
    line = Polyline(
        locations=coords,
        color="red",
        fill=False
    )
    m.add_layer(line)

In [58]:
m = Map(basemap=basemaps.Esri.WorldImagery, zoom=1, scroll_wheel_zoom=True, world_copy_jump=True)
m.layout.height = "510px"

options = [f"{city.city} ({city.iso3})" for _, city in df.iterrows()]
cityList = widgets.Combobox(
    options=options,
    placeholder='ex. London',
    description='City:',
    ensure_option=True,
    disabled=False
)

output = widgets.Output()

display(m, cityList, output)

def callback(change):
    with output:
        print(change["new"])

cityList.observe(callback, names="value")

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Combobox(value='', description='City:', ensure_option=True, options=('Tokyo (JPN)', 'Jakarta (IDN)', 'Delhi (I…

Output()

In [12]:
eastGraph = Graph()
for i in df.index:
    n = tree.query(coords[i:i+1], k=4)[1][0][1:]
    
    if (float(df.iloc[i].lng) < float(df.iloc[n[0]].lng)) or ((float(df.iloc[i].lng) > 0) and (float(df.iloc[n[0]].lng) < 0) and (float(df.iloc[i].lng)-float(df.iloc[n[0]].lng) >= 180)):
        eastGraph.add_edge(i, n[0], distanceTime(df.iloc[i], df.iloc[n[0]], 0))
    if (float(df.iloc[i].lng) < float(df.iloc[n[1]].lng)) or ((float(df.iloc[i].lng) > 0) and (float(df.iloc[n[1]].lng) < 0) and (float(df.iloc[i].lng)-float(df.iloc[n[1]].lng) >= 180)):
        eastGraph.add_edge(i, n[1], distanceTime(df.iloc[i], df.iloc[n[1]], 1))
    if (float(df.iloc[i].lng) < float(df.iloc[n[2]].lng)) or ((float(df.iloc[i].lng) > 0) and (float(df.iloc[n[2]].lng) < 0) and (float(df.iloc[i].lng)-float(df.iloc[n[2]].lng) >= 180)):
        eastGraph.add_edge(i, n[2], distanceTime(df.iloc[i], df.iloc[n[2]], 2))

In [13]:
startNode = 34
dijkstraEast = DijkstraSPF(eastGraph, startNode)
dijkstraEast.get_distance(4897)

inf

In [14]:
for startNode in df.index:
    for candidate in dijkstraEast.get_adjacent_nodes(eastGraph, startNode):
        dijkstraEast = DijkstraSPF(eastGraph, candidate)
        if dijkstraEast.get_distance(startNode) != float('inf'):
            print(f"{candidate}\t{startNode}\t{dijkstraEast.get_path(startNode)}")

26375	7355	[26375, 7355]
26342	7355	[26342, 26375, 7355]
26390	7355	[26390, 26375, 7355]
26390	26342	[26390, 26375, 26342]
26375	26342	[26375, 26342]
7355	26375	[7355, 26375]
26342	26375	[26342, 26375]
26375	26390	[26375, 26342, 26390]


In [15]:
add_marker(df.iloc[26390])
add_marker(df.iloc[26375])
add_marker(df.iloc[7355])

In [16]:
line = Polyline(
    locations=[
        [df.iloc[123].lat, df.iloc[123].lng],
        [df.iloc[26375].lat, df.iloc[26375].lng],
        [df.iloc[7355].lat, df.iloc[7355].lng],
        [df.iloc[123].lat, df.iloc[123].lng]
    ],
    color="red",
    fill=False
)

m.add_layer(line)

In [44]:
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()